# Partitioning Strategies

This notebook demonstrates various table partitioning strategies in PostgreSQL:
* Range partitioning
* List partitioning
* Hash partitioning
* Partition maintenance

## 1. Range Partitioning by Date

In [ ]:
-- Create partitioned orders table by date range
CREATE TABLE orders_partitioned (
    order_id SERIAL,
    customer_id INT,
    order_date TIMESTAMP,
    ship_date TIMESTAMP,
    ship_mode VARCHAR(20),
    status VARCHAR(20),
    total_amount DECIMAL(10,2),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
) PARTITION BY RANGE (order_date);

-- Create partitions
CREATE TABLE orders_2020 PARTITION OF orders_partitioned
    FOR VALUES FROM ('2020-01-01') TO ('2021-01-01');

CREATE TABLE orders_2021 PARTITION OF orders_partitioned
    FOR VALUES FROM ('2021-01-01') TO ('2022-01-01');

CREATE TABLE orders_2022 PARTITION OF orders_partitioned
    FOR VALUES FROM ('2022-01-01') TO ('2023-01-01');

CREATE TABLE orders_2023 PARTITION OF orders_partitioned
    FOR VALUES FROM ('2023-01-01') TO ('2024-01-01');

In [ ]:
-- Insert data into partitioned table
INSERT INTO orders_partitioned
SELECT * FROM orders;

-- Query using partition pruning
EXPLAIN ANALYZE
SELECT *
FROM orders_partitioned
WHERE order_date BETWEEN '2022-01-01' AND '2022-12-31';

## 2. List Partitioning by Region

In [ ]:
-- Create partitioned customers table by country
CREATE TABLE customers_partitioned (
    customer_id SERIAL,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(100),
    country VARCHAR(50),
    city VARCHAR(50),
    postal_code VARCHAR(20),
    segment VARCHAR(20),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
) PARTITION BY LIST (country);

-- Create partitions
CREATE TABLE customers_usa PARTITION OF customers_partitioned
    FOR VALUES IN ('USA');

CREATE TABLE customers_canada PARTITION OF customers_partitioned
    FOR VALUES IN ('Canada');

CREATE TABLE customers_uk PARTITION OF customers_partitioned
    FOR VALUES IN ('UK');

CREATE TABLE customers_other PARTITION OF customers_partitioned
    DEFAULT;

In [ ]:
-- Insert data into partitioned table
INSERT INTO customers_partitioned
SELECT * FROM customers;

-- Query specific partition
EXPLAIN ANALYZE
SELECT *
FROM customers_partitioned
WHERE country = 'USA';

## 3. Hash Partitioning

In [ ]:
-- Create hash partitioned order_items table
CREATE TABLE order_items_partitioned (
    item_id SERIAL,
    order_id INT,
    product_id INT,
    quantity INT,
    unit_price DECIMAL(10,2),
    discount DECIMAL(4,2),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
) PARTITION BY HASH (order_id);

-- Create partitions
CREATE TABLE order_items_p0 PARTITION OF order_items_partitioned
    FOR VALUES WITH (MODULUS 4, REMAINDER 0);

CREATE TABLE order_items_p1 PARTITION OF order_items_partitioned
    FOR VALUES WITH (MODULUS 4, REMAINDER 1);

CREATE TABLE order_items_p2 PARTITION OF order_items_partitioned
    FOR VALUES WITH (MODULUS 4, REMAINDER 2);

CREATE TABLE order_items_p3 PARTITION OF order_items_partitioned
    FOR VALUES WITH (MODULUS 4, REMAINDER 3);

In [ ]:
-- Insert data into partitioned table
INSERT INTO order_items_partitioned
SELECT * FROM order_items;

-- Query with specific order_id
EXPLAIN ANALYZE
SELECT *
FROM order_items_partitioned
WHERE order_id = 1000;

## 4. Partition Maintenance

In [ ]:
-- Add new partition for future data
CREATE TABLE orders_2024 PARTITION OF orders_partitioned
    FOR VALUES FROM ('2024-01-01') TO ('2025-01-01');

-- Detach old partition
ALTER TABLE orders_partitioned DETACH PARTITION orders_2020;

-- Archive old partition
CREATE TABLE orders_archive AS
SELECT * FROM orders_2020;

-- Drop old partition
DROP TABLE orders_2020;

## Best Practices for Partitioning

1. **Choosing Partition Strategy**
   - Use RANGE for time-based or numerical ranges
   - Use LIST for categorical data
   - Use HASH for uniform distribution
   - Consider data access patterns

2. **Partition Size**
   - Keep partitions reasonably sized
   - Consider maintenance operations
   - Plan for data growth
   - Monitor partition usage

3. **Maintenance Operations**
   - Regular partition addition
   - Archive old partitions
   - Update statistics
   - Monitor space usage

4. **Common Pitfalls**
   - Too many small partitions
   - Poor partition key choice
   - Insufficient maintenance planning
   - Ignoring partition pruning